In [586]:
from importlib import reload

import DataProvider
import Utils

reload(DataProvider)
reload(Utils)

<module 'Utils' from '/home/vincent/vincent/sharedwork/data/market-hedging/Utils.py'>

In [536]:
df = DataProvider.get_random_securities(10, "2013-10-01", "2018-10-01")
df_spy = DataProvider.get_securities("SPY", "2013-10-01", "2018-10-01")

In [537]:
df = Utils.attach_return(df, "symbol")
df_spy = Utils.attach_return(df_spy, "symbol")

In [ ]:
df = df.reset_index().merge(
    df_spy.reset_index().drop("symbol", axis=1),
    on="date",
    suffixes=["", "_spy"]
).dropna().set_index(df.index.names)

In [552]:
df.sample(10)

adjClose    return  adjClose_spy  return_spy
symbol date                                                      
PAR    2011-09-26    3.100000  0.006494    100.886032    0.023780
CACC   2017-10-10  282.920000  0.013614    250.064447    0.002638
       2015-08-24  188.390000 -0.023127    177.656926   -0.040884
BOCH   2013-06-20    4.477488  0.000000    142.943678   -0.024778
GRAM   2014-03-21   16.840795 -0.046088    170.343380   -0.003848
PAR    2012-04-16    4.840000  0.018947    120.224418   -0.000656
QUMU   2016-05-25    3.720000  0.019178    199.399291    0.006783
KAR    2017-01-19   43.187399 -0.013086    218.749612   -0.003705
QUMU   2011-02-07   13.006591 -0.001392    112.903123    0.006252
BOCH   2013-02-05    4.505733  0.018367    134.855380    0.010131

In [588]:
df = Utils.attach_beta(df, "return_spy", "return", "symbol", 10)

In [589]:
df.sample(10)

,,adjClose,return,adjClose_spy,return_spy,beta
symbol,date,,,,,
KAR,2012-04-19,14.305526,0.003513,120.812162,-0.006421,1.321060
QUMU,2014-08-05,13.760000,-0.017143,176.498340,-0.009696,-0.095350
PAR,2013-09-19,4.620000,0.026667,155.741655,-0.001682,0.779021
SOHU,2013-06-13,63.810000,-0.004524,147.257098,0.015209,0.966549
PAR,2017-09-01,10.680000,-0.006512,242.205780,0.001414,-0.422827
NEP,2018-02-21,37.944923,-0.035661,266.563164,-0.004974,0.964491
DPZ,2011-04-04,15.987318,-0.001624,114.500501,0.000826,-0.428903
PAYX,2011-04-04,24.599336,0.007540,114.500501,0.000826,-0.956516
CACC,2011-04-14,74.370000,-0.009852,113.039816,0.000761,3.779681
